In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import umap
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import random
from sklearn.metrics import silhouette_score
import seaborn as sns
import traceback

In [8]:
from const import action_map

In [9]:
action_map['2']


'subject_nose_poke_side'

In [10]:
import os
import os.path as osp
os.getcwd()

'D:\\Project- Electro\\Predict_Social_Behavior\\src\\correct_simaba_files'

In [11]:
# df_path = 'cropped_Rat1 probe 4-day2-free_2019-05-13-114301-0000.csv'
df_path = 'cropped_Rat10-probe13-day1-free_2018-07-15-103307-0000.csv'
folder_path = 'samples'
df_full_path = osp.join(folder_path,df_path)
df = pd.read_csv(df_full_path)
all_joints = [a for a in df.columns if any(substring in a for substring in ['_x', '_y'])]
df_pose = df[all_joints]

In [74]:
import cv2
def load_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    return frames

def generate_distinct_colors(num_groups, num_classes):
    # Calculate the base hue range for each group
    base_hues = np.linspace(0, 1, num_groups + 1)[:-1]

    # Calculate the class hue offset within each group
    class_hue_offsets = np.linspace(0, 1, num_classes + 1)[:-1] / (num_classes * 0.25)

    # Generate distinct RGB colors for each group and class
    colors = []
    for base_hue in base_hues:
        group_colors = []
        for hue_offset in class_hue_offsets:
            hue = (base_hue + hue_offset) % 1
            rgb_color = plt.cm.hsv(hue)
            group_colors.append(rgb_color[:3])  # Keep only RGB channels
        colors.append(group_colors)
    return colors

    
def draw_circles_with_text(frame, positions, colors, texts=None, text_color=(255, 255, 255),
                           radius=10, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=0.5, thickness=1):
    # Convert frame to RGB format (OpenCV uses BGR by default)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    for ind, ((x, y), color) in enumerate(zip(positions, colors)):
        # Convert the color from 0-255 range to tuple format (BGR)
        b, g, r = color
        x = int(x)
        y = int(y)
        circle_color = (b*255, g*255, r*255)
        
        
        # Draw circle on the frame
        cv2.circle(frame_rgb, (x, y), radius, circle_color, -1)
        
        if texts is not None:
            text_pos = (x - radius, y - radius - 10)
            # Draw text on the frame
            text = texts[ind]
            
            frame_bgr = cv2.putText(frame_rgb, text, text_pos, font,
                        font_scale, text_color, thickness, lineType=cv2.LINE_AA)


          
    # Convert frame back to BGR format for display
    frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
    return frame_bgr
    

    
def display_label_on_video(frame, index, labels_array,new_label=None,
                           display_label=True, display_map_flag=True, display_label_location=None, save_label_flag=False):
    height, width = frame.shape[:2]
       
        
    if not new_label is None and is_char_keycode(new_label):
        new_label = chr(new_label)            

        if new_label in action_map:
            labels_array[index] = new_label
            
    else:
        if index > 0 and save_label_flag:
            labels_array[index] = labels_array[index-1]
        
    
    if display_label_location is None:
        display_label_location = (width//2, height//2 + 100)
    
    label = labels_array[index]    
    if display_label:
        label = action_map[label]
        
    cv2.putText(frame, f"Cluster: {label}", (width//2, height//2 + 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    h = 160
    if display_map_flag:
                
        h_increment = 30
        for key, val in action_display.items():
            if key == 'save mode':
                continue
            cv2.putText(frame, f"{key}: {val}", (20,h), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            h += h_increment
    cv2.putText(frame, f"Save mode: {action_display['save mode']}", (20,h), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    return labels_array
    
def is_char_keycode(key_code):
    # Check if the key code corresponds to a printable ASCII character
    return 32 <= key_code <= 126
    
    
def play_clustered_video(frames_in, df_pose, columns_to_plot,
                         labels_array=None, additional_variables=None, 
                         show_class=None, enable_labeling=False):
    
    try:
        action_display['save mode'] = False
        abs_fps = 29
        frame = frames_in[0].copy()
        height, width = frame.shape[:2]

        pos_1_x = df_pose[[a for a in columns_to_plot if '1' in a and '_x' in a]].values
        pos_1_y = df_pose[[a for a in columns_to_plot if '1' in a and '_y' in a]].values

        pos_2_x = df_pose[[a for a in columns_to_plot if '2' in a and '_x' in a]].values
        pos_2_y = df_pose[[a for a in columns_to_plot if '2' in a and '_y' in a]].values

        colnames_1_x =   [a for a in columns_to_plot if '1' in a and '_x' in a]
    #     colnames_1_y =   [a for a in df_pos.columns if '1' in a and '_y' in a]

        colnames_2_x =   [a for a in columns_to_plot if '2' in a and '_x' in a]
    #     colnames_2_y =   [a for a in df_pos.columns if '2' in a and '_y' in a]


        colors = generate_distinct_colors(2, len(colnames_1_x))


        frame_number = 0
        n_frames = len(frames)
        pause_vid = False

        pause_fliker_counter = 0
        fps = abs_fps

        display_map_flag = True
        save_label_flag = action_display['save mode']

        prev_key = -1
        continuous_presss = 1
        continuous_presss_factor = 1
        
        while frame_number < len(frames_in):

            # Get user input

            key = cv2.waitKeyEx(int(1000 / fps))  # Display each frame for the appropriate time

            if key == prev_key and key !=-1 and not pause_vid:
                continuous_presss += 1
            else:
                continuous_presss = 1

            if key !=-1:
                if is_char_keycode(key):
                    print(chr(key))
                else:
                    print(key)
            
            

            if continuous_presss==1:
                if key ==  ord('d'):
                    display_map_flag = not display_map_flag

                elif key == ord('s'):
                    action_display['save mode'] = not action_display['save mode']
                    save_label_flag = action_display['save mode']

            continuous_presss_factor = 1+continuous_presss/fps*0.5


            prev_key = key
            
            shift_size = int(fps*continuous_presss)
            
            if key == 27:  # Press 'Esc' to exit
                break
            elif key == 2424832: # Press 'Left Arrow' to go back 10 frames
                frame_number = np.max([0, frame_number-shift_size])
            elif key == 2555904:               
                frame_number = np.min([n_frames, frame_number+shift_size]) # Press 'Right Arrow' to skip 10 frames
            elif key == 2490368 : # Predd "up" to speed up
                fps = np.min([100, fps+int(1*continuous_presss)])
            elif key == 2621440: # Predd "down" to slow down
                fps = np.max([1, fps-int(1*continuous_presss)])
            elif key == ord('r'): # Predd press "r" to reset
                fps = abs_fps

            elif key == 32: # Pasue video
                pause_vid = not pause_vid
                pause_fliker_counter = 0

            
            frame_number = 0 if frame_number<0 else frame_number
            frame = frames_in[frame_number].copy()

            
                        
            labels_array = display_label_on_video(frame, frame_number, labels_array,new_label=key,
                                   display_label=True, display_map_flag=display_map_flag, display_label_location=None,
                                  save_label_flag=save_label_flag and not pause_vid)       



            cluster = labels_array[frame_number]
            time = frame_number / abs_fps

            if show_class is not None:
                while show_class != cluster and frame_number<n_frames:
                    frame_number += 1
                    cluster = cluster_labels[frame_number]

            if frame_number > n_frames:
                break      

            cv2.putText(frame, f"fps: {fps}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(frame, f"Frame: {frame_number}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(frame, f"Time: {time:.2f} s", (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    #         cv2.putText(frame, f"Cluster: {cluster}", (width//2, height//2 + 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            if continuous_presss>1:
                cv2.putText(frame, f"Factor: {continuous_presss_factor}",
                            (20, 600), cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 0, 255), 2)
            if additional_variables:
                h = 160
                for var in additional_variables:
                    value = np.round(df_pose.iloc[frame_number][var])

                    cv2.putText(frame, f"{var}: {value:0.0f}", (150, h), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    h += 30

            # Draw Pose

            x = pos_1_x[frame_number,:]
            y = pos_1_y[frame_number,:]
            pos_1 = list(zip(x,y))
            frame = draw_circles_with_text(frame, pos_1, colors[0], texts=colnames_1_x)

            x = pos_2_x[frame_number,:]
            y = pos_2_y[frame_number,:]
            pos_2 = list(zip(x,y))
            frame = draw_circles_with_text(frame, pos_2, colors[1], texts=colnames_2_x)


            # Display
            cv2.imshow("Clustered Video", frame)
            
            if pause_vid:
                frame_temp = frame.copy()
                cv2.putText(frame_temp, f"Video Paused", (width//2, height//2), cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 0, 255), 2)

                if pause_fliker_counter%50 <25:
                    cv2.imshow("Clustered Video", frame_temp)
                else:
                    cv2.imshow("Clustered Video", frame)
                pause_fliker_counter +=1
            else:
                frame_number+=1

            if cv2.getWindowProperty("Clustered Video", cv2.WND_PROP_VISIBLE) <1:
                break
        cv2.destroyAllWindows()
    except Exception as e:
        traceback_str = traceback.format_exc()
        print(f"An error occurred: {traceback_str}")
    finally:
        cv2.destroyAllWindows()
        return labels_array
    return labels_array
    


In [59]:
# frames = load_video_frames(video_path)

In [44]:
# video_path = 'cropped_Rat1 probe 4-day2-free_2019-05-13-114301-0000.avi/'
video_path = df_full_path.replace('csv','avi')
if 'frames' not in globals():
    frames = load_video_frames(video_path)

In [45]:
consol_actions = {'r':'Reset fps',
                       'd':'Toggle display',
                        's':'Save labels',
                        'save mode': False,
                       'escape':'Close windo'}
action_display = action_map.copy()
action_display.update(consol_actions)

                       

In [46]:
labels_array = ['7']*len(frames)

In [47]:
# # labels = smooth_time_series(labels, x=30, y=500)
# joints_to_plot = [a for a in all_joints if '_derivative' not in a]

# labels_array = play_clustered_video(frames_in=frames, df_pose=df_pose, columns_to_plot=all_joints,
#                          labels_array=labels_array, additional_variables=None, 
#                          show_class=None, enable_labeling=True)


In [48]:
# Get training data
# Fix missing or outlier joints


all_joints = [a for a in df.columns if any(substring in a for substring in ['_x', '_y']) and '_1' in a]
prob_colums = [a.replace('x','p') for a in all_joints if '_x' in a]
all_column_names = [a for sub in [all_joints, prob_colums] for a in sub]
df_model = df[[a for a in df.columns if a in all_column_names]]

feature_columns = [a for a in df_model.columns if 'nose' not in a.lower() and '_p' not in a]
target_columns = [a for a in df_model.columns if 'nose' in a.lower() and '_p' not in a]
prob_colums = [a for a in prob_colums if 'nose' in a.lower()]


In [19]:
df_model = df[feature_columns+target_columns]
zscore_df = df_model.copy()
scalers = {}
for c in df_model.columns:
    scalers[c] = StandardScaler()
    scalers[c].fit(df[c].values.reshape(-1, 1))
    zscore_df[c] = scalers[c].transform(df[c].values.reshape(-1, 1)).reshape(-1,)
    
zscore_df

,Ear_left_1_x,Ear_left_1_y,Ear_right_1_x,Ear_right_1_y,Center_1_x,Center_1_y,Lat_left_1_x,Lat_left_1_y,Lat_right_1_x,Lat_right_1_y,Tail_base_1_x,Tail_base_1_y,Tail_end_1_x,Tail_end_1_y,Nose_1_x,Nose_1_y
0,0.728378,-0.146983,0.981243,-0.134178,0.884503,-0.042547,1.111471,0.247764,1.191531,-0.151023,1.329112,0.595739,1.413205,0.320700,1.237468,-0.273276
1,0.721078,-0.144689,0.976481,-0.134054,0.880224,-0.023683,1.110610,0.245110,1.191902,-0.150073,1.459815,0.631912,1.416216,0.317361,1.238079,-0.269163
2,0.722967,-0.138545,0.975530,-0.131546,0.872505,-0.030189,1.111479,0.244714,1.194113,-0.146350,1.329775,0.595753,1.407734,0.313190,1.255065,-0.275677
3,0.728836,-0.139611,0.978261,-0.127183,0.873299,-0.019547,1.112320,0.203966,1.195896,-0.149617,1.329044,0.595531,1.412862,0.321702,1.254314,-0.273521
4,0.728503,-0.137216,0.975465,-0.129979,0.873111,-0.013280,1.111866,0.247133,1.191563,-0.144992,1.361654,0.595965,1.410494,0.318962,1.255550,-0.266850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10452,-2.888459,-1.426906,-2.835135,-1.385258,-2.894685,-1.408728,-2.951274,-1.409935,-3.016088,-1.417533,-2.912291,-1.305454,-2.791281,-1.475182,-2.326207,-1.205948
10453,-2.888459,-1.426906,-2.835135,-1.385258,-2.894685,-1.408728,-2.951274,-1.409935,-3.016088,-1.417533,-2.912291,-1.305454,-2.791281,-1.475182,-2.326207,-1.205948
10454,-2.888459,-1.426906,-2.835135,-1.385258,-2.894685,-1.408728,-2.951274,-1.409935,-3.016088,-1.417533,-2.912291,-1.305454,-2.791281,-1.475182,-2.326207,-1.205948
10455,-2.888459,-1.426906,-2.835135,-1.385258,-2.894685,-1.408728,-2.951274,-1.409935,-3.016088,-1.417533,-2.912291,-1.305454,-2.791281,-1.475182,-2.326207,-1.205948


In [20]:
scalers['Ear_left_1_x'].inverse_transform(np.array([0.1,0.3]).reshape(2,-1))

array([[763.88108381],
       [815.0031625 ]])

In [21]:
df_features = zscore_df[feature_columns]
df_target = zscore_df[target_columns]
df_prob = df[prob_colums]

In [24]:
# create a dl model to predict the location of each joint based on the other joints
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F


class MissingJointPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout_rate=0.2):
        super(MissingJointPredictor, self).__init__()
        layers = []
        layers.append(nn.BatchNorm1d(input_size))
        layers.append(nn.Linear(input_size, hidden_size))
        init.xavier_uniform_(layers[-1].weight)  # Apply Xavier/Glorot initialization
        layers.append(nn.ReLU())
        layers.append(nn.BatchNorm1d(hidden_size))
        layers.append(nn.Dropout(dropout_rate))  # Apply dropout
        
        for _ in range(num_layers - 2):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.ReLU())
        
        layers.append(nn.Linear(hidden_size, output_size))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x, target,p):
        x =  self.network(x)
        
        squared_errors = F.mse_loss(x, target, reduction='none')
        squared_errors = squared_errors * p
#         squared_errors = torch.clamp(squared_errors, min=0, max=100)
        loss = squared_errors.mean()
#         print(loss)
        return loss, x

# Set parameters
input_size = len(feature_columns)  # Number of data points
hidden_size = 100
output_size = 2  # Assuming 2D coordinates
num_layers = 3  # Number of layers

# Create an instance of the model with the specified number of layers
model = MissingJointPredictor(input_size, hidden_size, output_size, num_layers)

# Loss function and optimizer
# criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.SGD(model.parameters(), lr=0.01)


# Apply L2 regularization through gradient clipping
l2_lambda = 0.01  # Adjust the regularization strength
clip_value = 0.5  # Adjust the maximum gradient value for clipping


# Assuming `X` is your input data and `y` is your target (missing joint coordinates)

# Convert data to PyTorch tensors
X = df_features.values
y = df_target.values
p = df_prob.values

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
p = torch.tensor(p, dtype=torch.float32)

# Training loop
epochs = 1000

for epoch in range(epochs):
    # Forward pass
    loss, pred = model(X,y,p)
    
    
    # Compute the loss
#     loss = criterion(outputs, y)
    
    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    for param in model.parameters():
        if param.grad is not None:
            param.grad.data += l2_lambda * param.data  # Add L2 regularization term
            param.grad.data = torch.clamp(param.grad.data, -clip_value, clip_value)  # Clip gradients
        optimizer.step()  # Update model parameters
    
    
    if epoch % 50 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# # Now you can use the trained model to predict missing joint coordinates
# new_data = torch.tensor(new_data, dtype=torch.float32)
# predicted_joint = model(new_data)
# print(f'Predicted Missing Joint: {predicted_joint}')

Epoch [1/1000], Loss: 0.7524
Epoch [51/1000], Loss: 0.2787
Epoch [101/1000], Loss: 0.2432
Epoch [151/1000], Loss: 0.2202
Epoch [201/1000], Loss: 0.2104
Epoch [251/1000], Loss: 0.1998
Epoch [301/1000], Loss: 0.1821
Epoch [351/1000], Loss: 0.1703
Epoch [401/1000], Loss: 0.1626
Epoch [451/1000], Loss: 0.1607
Epoch [501/1000], Loss: 0.1516
Epoch [551/1000], Loss: 0.1467
Epoch [601/1000], Loss: 0.1493
Epoch [651/1000], Loss: 0.1395
Epoch [701/1000], Loss: 0.1361
Epoch [751/1000], Loss: 0.1357
Epoch [801/1000], Loss: 0.1273
Epoch [851/1000], Loss: 0.1404
Epoch [901/1000], Loss: 0.1230
Epoch [951/1000], Loss: 0.1214


In [25]:
corrected_x_nose = scalers['Nose_1_x'].inverse_transform(pred.detach().numpy()[:,0].reshape(1,-1)).reshape(-1,1)
corrected_y_nose = scalers['Nose_1_y'].inverse_transform(pred.detach().numpy()[:,1].reshape(1,-1)).reshape(-1,1)

In [30]:
corrected_coordinates_columns = ['Nose_1_x_corrected','Nose_1_y_corrected']

In [75]:
df_pose = df_pose.copy()
df_pose['Nose_1_x_corrected'] = corrected_x_nose
df_pose['Nose_1_y_corrected'] = corrected_y_nose
all_joints = all_joints+corrected_coordinates_columns

# labels = smooth_time_series(labels, x=30, y=500)
# joints_to_plot = [a for a in all_joints if '_derivative' not in a]

labels_array = play_clustered_video(frames_in=frames, df_pose=df_pose, columns_to_plot=all_joints,
                         labels_array=labels_array, additional_variables=None, 
                         show_class=None, enable_labeling=True)

2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
d
2424832
2424832
2555904
2555904
2555904
2555904
2555904
2555904
2555904
2555904
27


In [73]:
labels_array = play_clustered_video(frames_in=frames, df_pose=df_pose, columns_to_plot=[],
                         labels_array=labels_array, additional_variables=None, 
                         show_class=None, enable_labeling=True,
                                    corrected_coordinates_columns=corrected_coordinates_columns)

[(834.2212, 178.8655)]
[(1003.27045, 241.4709)]
[(829.9961, 232.85611)]
[(828.3464, 232.1292)]
[(994.2225, 231.44229)]
[(802.0328, 147.57928)]
[(910.19885, 201.89964)]
[(947.59247, 265.87408)]
[(899.65546, 235.05878)]
[(892.78687, 204.6239)]
[(862.4258, 199.23961)]
[(921.2938, 212.98103)]
[(797.5097, 219.7305)]
[(892.28723, 231.69983)]
[(925.6305, 263.36038)]
[(964.2495, 233.70953)]
[(730.69965, 184.89438)]
[(892.2875, 236.44287)]
[(915.3797, 198.83714)]
[(929.6573, 235.21959)]
[(949.87994, 224.18317)]
[(860.64435, 218.46991)]
[(938.6527, 246.6825)]
[(895.85315, 227.69643)]
[(972.59344, 228.69809)]
[(929.6301, 213.11116)]
[(940.7151, 179.03207)]
[(938.3512, 208.17146)]
[(943.92224, 240.13585)]
[(918.48157, 215.88351)]
[(986.5735, 224.47678)]
[(936.06854, 208.42932)]
[(963.2588, 289.08243)]
[(923.6018, 206.19373)]
[(880.2736, 240.72517)]
[(927.5735, 235.1245)]
[(995.1542, 195.21808)]
[(958.88135, 222.24747)]
[(987.42206, 220.15334)]
[(938.50385, 202.08887)]
[(956.64795, 193.53963)]
[(99

In [72]:
np.max(df_pose.values)

1310.47998046875